# **WINE CASE: Modeling**

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
wine_db = sqlite3.connect('wine.db')
cursor = wine_db.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('ficha_tecnica',), ('harmonizacao',), ('price',), ('review_rating',), ('sommelier',)]


In [3]:
ficha_df = pd.read_sql_query("SELECT * from ficha_tecnica", wine_db)

In [4]:
ficha_df.head()

,sku,tipo,uva,pais,regiao,vinicola,teor_alcoolico,amadurecimento,classificacao,visual,aroma,gosto,temperatura,potencial_de_guarda
0,10839,Espumante,"Chardonnay, Glera, Pinot Bianco",Itália,Friuli,Fantinel,11.50% ABV,Em tanques de aço inox,Brut,Amarelo-palha com reflexos esverdeados,"Maçã, flores brancas","Leve, fresco, floral, frutado, acidez agradável",8 °C,4 anos
1,11549,Branco,Furmint (100%),Hungria,Tokaj,Pajzos,11.00% ABV,Sem passagem por carvalho.,Suave/Doce,Amarelo dourado,"Frutas brancas maduras, mel, figo seco, frutas...","Doce, equilibrado, acidez agradável, final longo",10 °C,15 anos
2,11610,Espumante,"Chenin Blanc (60%), Chardonnay (40%)",Argentina,Mendoza,Finca La Chamiza,12.00% ABV,1 mês sobre as borras.,Brut,Amarelo,"Frutas amarelas maduras, toque de fermento","Maduro, leve, acidez agradável, delicada cremo...",8 °C,3 anos
3,11982,Licoroso,Nero d'Avola,Itália,Marsala,Cantine Pellegrino,18.00% ABV,Mais de um ano em barricas,Suave/Doce,Rubi com reflexos âmbar.,"Frutas negras em compota, frutas secas, carame...","Levemente adocicado, taninos leves e sabor int...",12 °C,6 anos
4,11983,Licoroso,"Catarratto, Grillo",Itália,Marsala,Cantine Pellegrino,18.00% ABV,4 anos em barris de carvalho.,Suave/Doce,Ouro intenso com reflexos âmbar.,"Frutas cristalizadas, casca de laranja, baunil...","Doce, macio, cheio e persistente.",12 °C,8 anos


## **Data pre-processing**

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [6]:
def OneHotEncoder_to_DataFrame(category_list, index_list):
    
    from sklearn.preprocessing import LabelEncoder
    from sklearn.preprocessing import OneHotEncoder
    
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(category_list)
    label_enconded = label_encoder.inverse_transform(np.arange(len(np.unique(integer_encoded))))
    
    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    
    category_df = pd.DataFrame(onehot_encoded, index=index_list, columns=label_enconded)
    
    return category_df    

### **Tipos de vinhos**

In [7]:
tipos = ficha_df['tipo'].copy().values
tipos_df = OneHotEncoder_to_DataFrame(tipos, ficha_df.index)
tipos_df.head()

,Branco,Espumante,Frisante,Licoroso,Rosé,Tinto
0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0


### **Tipos de uvas**

In [8]:
uva_df = pd.DataFrame(index=ficha_df.index)
uva_df['Uvas'] = pd.Series(np.zeros(len(uva_df.index)))
uva_df['Nº de uvas'] = pd.Series(np.zeros(len(uva_df.index)))

In [9]:
import re
for i in ficha_df.index:
    uvas = ficha_df['uva'][i]
    uva_list = (re.sub(r" ?\([^)]+\)", "", uvas))
    uva_df['Uvas'][i] = uva_list
    uva_df['Nº de uvas'][i] = uva_list.count(',') + 1

/home/anderson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/anderson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [10]:
uva_df.head()

,Uvas,Nº de uvas
0,"Chardonnay, Glera, Pinot Bianco",3.0
1,Furmint,1.0
2,"Chenin Blanc, Chardonnay",2.0
3,Nero d'Avola,1.0
4,"Catarratto, Grillo",2.0


In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
max_features = 20
vectorizer = CountVectorizer(lowercase=False, ngram_range=(1,1), max_features=max_features)
bag_sparse = vectorizer.fit_transform(uva_df['Uvas'])
bag_of_words = pd.DataFrame.sparse.from_spmatrix(bag_sparse, columns=vectorizer.get_feature_names())
print(bag_of_words.keys())

Index(['Blanc', 'Cabernet', 'Carménère', 'Chardonnay', 'Franc', 'Garnacha',
       'Grenache', 'Malbec', 'Merlot', 'Meunier', 'Nacional', 'Noir', 'Petit',
       'Pinot', 'Sangiovese', 'Sauvignon', 'Syrah', 'Tempranillo', 'Touriga',
       'Verdot'],
      dtype='object')


In [13]:
uva_df.drop(columns='Uvas', inplace=True)
uva_df = pd.concat([uva_df, bag_of_words], axis=1)
uva_df.head()

,Nº de uvas,Blanc,Cabernet,Carménère,Chardonnay,Franc,Garnacha,Grenache,Malbec,Merlot,...,Nacional,Noir,Petit,Pinot,Sangiovese,Sauvignon,Syrah,Tempranillo,Touriga,Verdot
0,3.0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2.0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### **Vinhos por países**

In [27]:
paises = ficha_df['pais'].copy().values
pais_df = OneHotEncoder_to_DataFrame(paises, ficha_df.index)
pais_df.head()

,Alemanha,Argentina,Austrália,Brasil,Chile,China,Espanha,Estados Unidos,França,Hungria,Itália,Nova Zelândia,Portugal,Uruguai,África do Sul
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


### **Teor alcoolico**

In [17]:
teor_alcool = []

for teor_string in ficha_df['teor_alcoolico']:
        teor_alcool.append(teor_string.strip('% ABV'))
        
teor_alcool = np.array(teor_alcool, 'float')

In [18]:
teor_df = pd.DataFrame(index=ficha_df.index)
teor_df['Teor alcoolico'] = teor_alcool
teor_df.head()

,Teor alcoolico
0,11.5
1,11.0
2,12.0
3,18.0
4,18.0


### **Temperatura**

In [19]:
temp = []

for temp_string in ficha_df['temperatura']:
        temp.append(temp_string.strip(' °C'))
        
temp = np.array(temp, 'float')

In [20]:
temp_df = pd.DataFrame(index=ficha_df.index)
temp_df['Temperatura'] = temp
temp_df.head()

,Temperatura
0,8.0
1,10.0
2,8.0
3,12.0
4,12.0


In [21]:
classificacao = ficha_df['classificacao']
class_df = OneHotEncoder_to_DataFrame(classificacao, ficha_df.index)
class_df.head()

,Brut,Extra Brut,Meio Seco/Demi-Sec,Nature,Seco,Suave/Doce
0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,1.0


## **Model 1**

In [29]:
X_sample = pd.concat([tipos_df, uva_df, pais_df, teor_df, temp_df, class_df], axis=1)
X_sample.head()

,Branco,Espumante,Frisante,Licoroso,Rosé,Tinto,Nº de uvas,Blanc,Cabernet,Carménère,...,Uruguai,África do Sul,Teor alcoolico,Temperatura,Brut,Extra Brut,Meio Seco/Demi-Sec,Nature,Seco,Suave/Doce
0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0,0,0,...,0.0,0.0,11.5,8.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,...,0.0,0.0,11.0,10.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1,0,0,...,0.0,0.0,12.0,8.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0,0,0,...,0.0,0.0,18.0,12.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0,0,0,...,0.0,0.0,18.0,12.0,0.0,0.0,0.0,0.0,0.0,1.0


In [30]:
pot_anos = []

for pot_string in ficha_df['potencial_de_guarda']:
    if ('ano' not in pot_string):
        print('ERROR: potencial_de_guarda nao esta em anos:')
        print(pot_string)
    else:
        pot_anos.append(pot_string.strip('anos '))
        
pot_anos = np.array(pot_anos, 'int')

In [66]:
Y_sample = pd.DataFrame(index=ficha_df.index)
Y_sample['Potencial de guarda'] = pot_anos
Y_sample.head()

,Potencial de guarda
0,4
1,15
2,3
3,6
4,8


In [61]:
pot_range = [0,2,4,6,10,20,30,40,50]

In [62]:
Y_sample.groupby(pd.cut(Y_sample['Potencial de guarda'], pot_range)).sum()

,Potencial de guarda
Potencial de guarda,
"(0, 2]",31
"(2, 4]",1116
"(4, 6]",1512
"(6, 10]",1513
"(10, 20]",669
"(20, 30]",340
"(30, 40]",75
"(40, 50]",144


In [67]:
Y_sample['Potencial bin'] = pd.Series(np.zeros(len(Y_sample)))

In [70]:
for i in range(len(pot_range)-1):
    ii = (Y_sample['Potencial de guarda'] > pot_range[i]) & (Y_sample['Potencial de guarda'] <= pot_range[i+1])
    Y_sample['Potencial bin'][ii]

/home/anderson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [71]:
Y_sample

,Potencial de guarda,Potencial bin
0,4,1.0
1,15,4.0
2,3,1.0
3,6,2.0
4,8,3.0
...,...,...
848,3,1.0
849,5,2.0
850,5,2.0
851,6,2.0
